In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.metrics import classification_report

In [144]:
data_species = pd.read_csv('../Data/species.csv', index_col=0)

data = data_species.copy(deep=True)

#### Remove Duplicates

In [146]:
duplicates = ["REGION", "YEAR", "MAX_HARD_RELIEF", "PCT_CORAL"]

data = data.drop(duplicates, axis = 1)

#### Remove Features not related

In [148]:
not_related = ["no.divers", "model", "Random", "site"]

data = data.drop(not_related, axis = 1)

#### Remove Features not relevant for the model

In [150]:
#Most values are the same for Habitat_type_classLV0
print(data.Habitat_type_classLV0.value_counts())

not_for_model = ["Latitude", "Longitude", "Habitat_type_classLV0"]

data = data.drop(not_for_model, axis = 1)

Habitat_type_classLV0
Coral Reef and Hardbottom    3988
Unconsolidated Sediment         7
Seagrass                        4
Name: count, dtype: int64


## EDA

### Zones and Regions analysis

In [153]:
#Keys_divisions, Region

print(data.loc[data.Keys_Divisions.isna(),["Keys_Divisions", "Region"]])
print(data.loc[data.Keys_Divisions.isna(),["Keys_Divisions", "Region"]].Keys_Divisions.unique())
print(data.loc[data.Keys_Divisions.isna(),["Keys_Divisions", "Region"]].Region.unique())


#CONCLUSION: Kyes_Divisons and Region have the same information. The missing values of Keys_Divisions correspond to SEFCRI.

     Keys_Divisions  Region
1350            NaN  SEFCRI
1351            NaN  SEFCRI
1352            NaN  SEFCRI
1353            NaN  SEFCRI
1354            NaN  SEFCRI
...             ...     ...
3992            NaN  SEFCRI
3993            NaN  SEFCRI
3994            NaN  SEFCRI
3997            NaN  SEFCRI
3998            NaN  SEFCRI

[1429 rows x 2 columns]
[nan]
['SEFCRI']


In [154]:
# Fill missing values for categorical columns with a placeholder
data.fillna({'Keys_Divisions': 'SEFCRI'}, inplace=True)

In [155]:
#BNP, FKNMS, CoralECA, DryTortuga contain the same information combined in Keys_Divisions, so they can be dropped.

drop_individual_region = ["BNP", "FKNMS", "CoralECA", "DryTortugas"]

data = data.drop(drop_individual_region, axis = 1)

### Feature Interrelation Analysis

#### impact

In [158]:
print(data.corr(numeric_only=True).loc[:,["impact", "connectivity"]].sort_values(by = "impact", ascending = False, key = abs))

#CONLCUSION: The following features are highly related and maybe explainable only with the "impact" feature

# SST                            0.848783
# Recreational_fishermen_50km    0.832204
# Commercial_pounds_landed      -0.825649
# Marina_slips_25km              0.817805
# Population_50km                0.817650
# Population_20km                0.770920
# Tourist_fishing                0.741886
# Marina_slips_10km              0.690361
# connectivity                   0.670892

# ** Diversity_index **          0.142606


related_to_impact = ["SST", "Marina_slips_10km", "Marina_slips_25km", 
                     "Population_20km", "Population_50km", "Recreational_fishermen_50km",
                    "Tourist_fishing", "connectivity"]

#OBS: connectivity is also considerably related with the anthropogenic stressors' features


data = data.drop(related_to_impact, axis = 1)

                               impact  connectivity
impact                       1.000000      0.670892
SST                          0.848783      0.550070
Recreational_fishermen_50km  0.832204      0.729267
Commercial_pounds_landed    -0.825649     -0.640719
Marina_slips_25km            0.817805      0.681051
Population_50km              0.817650      0.745041
Population_20km              0.770920      0.687211
Tourist_fishing              0.741886      0.697488
Marina_slips_10km            0.690361      0.766698
connectivity                 0.670892      1.000000
NPP                         -0.663904     -0.556075
Pop_per_area_reef_20km       0.619547      0.650263
Deepwater                   -0.564464     -0.342929
SG_permits_50km              0.486797      0.175019
FSA                         -0.462780     -0.223802
Total_gravity_intercept      0.389253      0.409008
Total_gravity                0.389253      0.409008
Coral_cover                 -0.283849     -0.222864
Artificial_r

#### Depth, Depth_Sbrocco, Deepwater

In [160]:
feat = ["Depth", "Depth_Sbrocco", "Deepwater"]
print(data.corr(numeric_only = True).loc[["Diversity_index"], feat])

#Depth and Depth_Sbrocco have a non-negligible correlation with Diversity_index.

data = data.drop("Deepwater", axis = 1)

                    Depth  Depth_Sbrocco  Deepwater
Diversity_index -0.207735       0.121668   -0.03791


#### Comm_reliance, Comm_engagement, Rec_reliance and Rec_engagement

In [162]:
comm_rec = ["Comm_reliance", "Comm_engagement", "Rec_reliance", "Rec_engagement"]

data.corr(numeric_only = True).loc[:,comm_rec]

print(data["Comm_reliance"].value_counts().head(3))
print(data["Comm_engagement"].value_counts().head(3))
print(data["Rec_reliance"].value_counts().head(3))
print(data["Rec_engagement"].value_counts().head(3))


#CONCLUSION: Most of the values are the same. So they are a good candidate to not be part of the final model. (?)

data = data.drop(comm_rec, axis = 1)

Comm_reliance
-0.129    2344
-0.128     263
 0.347     201
Name: count, dtype: int64
Comm_engagement
-0.158    2344
-0.105     227
 2.960     201
Name: count, dtype: int64
Rec_reliance
-0.185    2647
 0.044     277
 0.995     201
Name: count, dtype: int64
Rec_engagement
-0.260    2647
-0.176     277
 3.101     201
Name: count, dtype: int64


##### Features unsure

In [164]:
"SG_charter_permits_25km", 
"SG_permits_50km",
"Coral_area_UFRTM_20km", 
"FSA", 
"HABITAT_CD",
"NPP",
"Wave_exposure", 
"Artificial_reefs_1km", 
"Year",
"Month"

'Month'

#### EDA above not yet finished (!)

#### Fill Missing Values

In [167]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
data = data.dropna(axis = 0)#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 

In [168]:
#(This will probably give bad results) - For the Coral_cover maybe will be better 
#to remove the rows all together, since we cannot infer by the location



# Handle missing values: fill missing numeric values with the median
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    data[col] = data[col].fillna(data[col].median())

#### Convert Categorical Data into Numeric

In [170]:
# Convert categorical columns to numeric using label encoding
non_numeric_cols = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in non_numeric_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [171]:
print("\n \n === Features Selected === \n\n")
print(data.columns)


 
 === Features Selected === 


Index(['Year', 'Month', 'Depth', 'Region', 'Coral_cover', 'Reef_complexity',
       'NPP', 'Wave_exposure', 'Habitat_type_classLV2',
       'Coral_area_UFRTM_20km', 'Coral_area_UFRTM_200km', 'Depth_Sbrocco',
       'FSA', 'Marine_reserve', 'Artificial_reefs_1km', 'SG_permits_50km',
       'SG_charter_permits_25km', 'Total_gravity_intercept', 'Total_gravity',
       'Keys_Divisions', 'Nursery_seagrass', 'Nursery_mangroves',
       'Commercial_pounds_landed', 'Pop_per_area_reef_20km', 'impact',
       'HABITAT_CD', 'Diversity_index'],
      dtype='object')


### Model

#### Linear

###### Linear Regression

In [175]:
# Prepare the data for the model
# Separate features and target
X = data.drop(columns=['Diversity_index'])
y = data['Diversity_index']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


reg_model = linear_model.LinearRegression()
reg_model.fit(X_train, y_train)

y_predict = reg_model.predict(X_test)

mae = mean_absolute_error(y_test, y_predict) #The lower the better
mse = mean_squared_error(y_test, y_predict) #The lower the better
r2 = r2_score(y_test, y_predict) #Closer to 1 better

print("MAE: ", mae) #Average error
print("MSE: ", mse) #Average error
print("R2: ", r2)  #How much the model explains the variance.

MAE:  0.0772990866975016
MSE:  0.013408166039528812
R2:  0.17768274894369718


#### Non-Linear

###### Random Forest Regressor

In [ ]:
from sklearn.metrics import r2_score

# Prepare the data for the model
# Separate features and target
X = data.drop(columns=['Diversity_index'])
y = data['Diversity_index']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train a Random Forest model
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')
r2 = r2_score(y_test, y_pred) #Closer to 1 better
print("R2: ", r2)  #How much the model explains the variance.